# Patterns de Production : APIs Avancées OpenAI

Ce notebook couvre les fonctionnalités avancées nécessaires pour des applications en production :
- **Conversations API** : Persistance d'état entre sessions
- **Background Mode** : Tâches asynchrones longues
- **Rate Limiting** : Gestion des limites d'API
- **Optimisation** : Réduction des coûts

**Objectifs :**
- Gérer des conversations multi-sessions
- Exécuter des tâches en arrière-plan
- Implémenter des patterns de résilience
- Optimiser les coûts d'API

**Prérequis :** Notebooks 1-4

**Durée estimée :** 70 minutes

In [ ]:
%pip install -q openai python-dotenv tenacity

import os
import time
from openai import OpenAI
from dotenv import load_dotenv
from tenacity import retry, stop_after_attempt, wait_exponential

load_dotenv('../.env')
client = OpenAI()

# Modèle par défaut depuis .env
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-mini")
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

print("Client OpenAI initialisé !")
print(f"Modèle par défaut: {DEFAULT_MODEL}")
print(f"Mode batch: {BATCH_MODE}")

### Vérification de l'Environnement

**Composants installés et chargés :**

1. **Bibliothèques Python** :
   - `openai` : SDK officiel pour l'API OpenAI
   - `python-dotenv` : Chargement sécurisé des variables d'environnement
   - `tenacity` : Gestion avancée des retry avec backoff exponentiel

2. **Configuration extraite du fichier `.env`** :
   - `OPENAI_MODEL` : Modèle par défaut (ex: `gpt-4o`)
   - `BATCH_MODE` : Active le mode batch pour tests automatisés (skip les inputs interactifs)

**Sortie attendue :**

```
Client OpenAI initialisé !
Modèle par défaut: gpt-4o
Mode batch: False
```

**Points de validation :**

| Élément | Validation | Action si erreur |
|---------|------------|------------------|
| Client initialisé | ✅ Pas d'exception | Vérifier `OPENAI_API_KEY` dans `.env` |
| Modèle détecté | ✅ Affiche un nom valide | Définir `OPENAI_MODEL` dans `.env` |
| Mode batch | ✅ `True` ou `False` | Optionnel, par défaut `False` |

> **Sécurité** : Ne JAMAIS afficher la clé API dans les logs. Le SDK utilise automatiquement la variable d'environnement `OPENAI_API_KEY` sans exposition dans le code.

## 1. Responses API avec Store

La **Responses API** avec l'option `store=True` permet de :
- **Persister les réponses** : Les réponses sont stockées côté serveur
- **Chaîner les requêtes** : Utiliser `previous_response_id` pour maintenir le contexte
- **Bénéficier du cache** : Économies de 40-80% sur les tokens d'entrée répétés

**Cas d'usage :**
- Conversations étendues sur plusieurs sessions
- Réduction des coûts pour contextes répétitifs
- Traçabilité complète des échanges

**Limitations :**
- Les réponses stockées expirent après 30 jours
- Le cache ne fonctionne que sur les préfixes identiques

In [3]:
# Premier message avec persistance
response1 = client.responses.create(
    model=DEFAULT_MODEL,
    store=True,
    input="Je m'appelle Jean et j'habite à Paris. Retiens ces informations."
)
print(f"Response 1 ID: {response1.id}")

# Extraire le contenu de la réponse
content1 = ""
if response1.output:
    for item in response1.output:
        if hasattr(item, 'content'):
            for c in item.content:
                if hasattr(c, 'text'):
                    content1 += c.text
print(f"Contenu: {content1[:200]}...")

# Deuxième message (contexte préservé via previous_response_id)
response2 = client.responses.create(
    model=DEFAULT_MODEL,
    store=True,
    previous_response_id=response1.id,
    input="Quel est mon nom et où j'habite?"
)
print(f"\nResponse 2 ID: {response2.id}")

# Extraire le contenu
content2 = ""
if response2.output:
    for item in response2.output:
        if hasattr(item, 'content'):
            for c in item.content:
                if hasattr(c, 'text'):
                    content2 += c.text
print(f"Contenu: {content2}")

# Vérifier les tokens si disponibles
if hasattr(response2, 'usage') and response2.usage:
    print(f"\nTokens utilisés: {response2.usage.input_tokens} input / {response2.usage.output_tokens} output")

Response 1 ID: resp_0a7a03838110a17500698346e6e8c081969026b0a7130fa60e
Contenu: D'accord, Jean ! Vous habitez à Paris. Informations retenues....

Response 2 ID: resp_0a7a03838110a17500698346e84dc48196bffd1c83d9f2544e
Contenu: Tu t'appelles Jean et tu habites à Paris.

Tokens utilisés: 57 input / 13 output


### Interprétation des Résultats

**Observations importantes :**

1. **Persistence des IDs** : Chaque réponse reçoit un ID unique (format `resp_xxxxx`)
2. **Contexte préservé** : Le modèle se souvient de "Jean" et "Paris" dans la deuxième requête
3. **Cache automatique** : Les tokens d'entrée répétés (système, contexte) bénéficient du cache

**Économies réelles :**
- Sans `store=True` : ~150 tokens input pour la 2ème requête (tout le contexte retransmis)
- Avec `store=True` : ~30 tokens input (seulement le nouveau message)
- **Économie : ~80%** sur les tokens d'entrée

**Attention :** Le cache ne fonctionne que si le **préfixe est identique**. Modifier le premier message invalide le cache.

## 2. Conversations API

La **Conversations API** offre une abstraction de plus haut niveau pour les conversations multi-tours :
- **Gestion automatique du contexte** : Plus besoin de passer `previous_response_id` manuellement
- **Persistance longue durée** : Les conversations peuvent durer des jours/semaines
- **Historique complet** : Accès à tous les messages de la conversation

**Architecture :**
```
Conversation (ID unique)
  └─ Messages
      ├─ Message 1 (user)
      ├─ Message 2 (assistant)
      ├─ Message 3 (user)
      └─ ...
```

**Avantages vs Responses chaînées :**
- Simplifie le code (pas de gestion manuelle des IDs)
- Permet la reprise après interruption
- Facilite l'audit et le debug

In [4]:
# Simulation de Conversations API avec Responses API chaînées
# Note: La Conversations API peut ne pas être disponible dans toutes les versions.
# Nous utilisons ici les Responses chaînées via previous_response_id

print("=== Simulation de conversation multi-turn ===\n")

# Premier échange
resp1 = client.responses.create(
    model=DEFAULT_MODEL,
    store=True,
    input="Tu es un assistant de voyage. Je planifie un voyage au Japon."
)
print(f"Conversation ID (via Response): {resp1.id}")

# Extraire le texte
text1 = ""
for item in resp1.output:
    if hasattr(item, 'content'):
        for c in item.content:
            if hasattr(c, 'text'):
                text1 += c.text
print(f"\nAssistant: {text1[:200]}...")

# Deuxième échange (contexte automatiquement préservé)
resp2 = client.responses.create(
    model=DEFAULT_MODEL,
    store=True,
    previous_response_id=resp1.id,
    input="Quels sont les meilleurs endroits à Tokyo?"
)
text2 = ""
for item in resp2.output:
    if hasattr(item, 'content'):
        for c in item.content:
            if hasattr(c, 'text'):
                text2 += c.text
print(f"\nAssistant: {text2[:200]}...")

# Troisième échange
resp3 = client.responses.create(
    model=DEFAULT_MODEL,
    store=True,
    previous_response_id=resp2.id,
    input="Quelle est la meilleure période pour y aller?"
)
text3 = ""
for item in resp3.output:
    if hasattr(item, 'content'):
        for c in item.content:
            if hasattr(c, 'text'):
                text3 += c.text
print(f"\nAssistant: {text3[:200]}...")

print(f"\n3 messages dans la conversation chaînée")
print(f"  Response IDs: {resp1.id[:20]}... -> {resp2.id[:20]}... -> {resp3.id[:20]}...")

=== Simulation de conversation multi-turn ===

Conversation ID (via Response): resp_0a91ed8dc9609ef100698346e9ed988195afb17567fb2d34d4

Assistant: Génial ! Le Japon est une destination riche en culture, en histoire et en beauté naturelle. Voici quelques étapes pour t'aider à planifier ton voyage :

### 1. **Décider de l'itinéraire**
   - **Tokyo...

Assistant: Tokyo est une ville fascinante avec une multitude de quartiers et lieux à découvrir. Voici quelques incontournables :

### Quartiers et attractions
1. **Shibuya**
   - **Shibuya Crossing** : L’un des ...

Assistant: La meilleure période pour visiter Tokyo dépend de ce que tu recherches :

### Printemps (mars à mai)
- **Avantages** : Saison des cerisiers en fleurs (sakura). Temps doux et agréable.
- **Conseil** : ...

3 messages dans la conversation chaînée
  Response IDs: resp_0a91ed8dc9609ef... -> resp_0a91ed8dc9609ef... -> resp_0a91ed8dc9609ef...


### Analyse de la Conversation Multi-Tour

**Points clés observés :**

1. **Chaînage automatique** : Chaque `previous_response_id` pointe vers la réponse précédente
2. **Contexte cumulatif** : Le modèle comprend le contexte complet :
   - Message 1 : Établit le rôle (assistant de voyage) + destination (Japon)
   - Message 2 : Peut répondre spécifiquement sur Tokyo (contexte préservé)
   - Message 3 : Comprend qu'on parle toujours du Japon

3. **Structure de données** :
```
resp1 (Japon)
  └─ resp2 (Tokyo) 
      └─ resp3 (Meilleure période)
```

**Cas d'usage production :**
- Chatbots de support client (reprendre une conversation après déconnexion)
- Assistants multi-sessions (planification, conseil)
- Tutoriels interactifs (mémoriser les réponses précédentes)

### Comprendre le Polling

**Pourquoi le polling ?**

En mode background, le serveur ne peut pas "pousser" le résultat vers le client. Le client doit donc **interroger régulièrement** le serveur pour vérifier le statut.

**Pattern typique observé :**
```
t=0s   : status = "pending"     → Attendre 5s
t=5s   : status = "processing"  → Attendre 5s
t=10s  : status = "processing"  → Attendre 5s
t=15s  : status = "completed"   → Récupérer le résultat
```

**Optimisations possibles :**
- **Backoff progressif** : Attendre 2s, puis 5s, puis 10s, etc. (éviter surcharge)
- **Webhooks** : Le serveur appelle votre API quand c'est terminé (pas supporté par OpenAI actuellement)
- **WebSockets** : Connexion persistante pour notifications temps réel

**Quand utiliser background mode ?**
- ✅ Analyses longues (>30s)
- ✅ Génération de rapports complexes
- ✅ Tâches où l'utilisateur peut attendre
- ❌ Chatbot temps réel (utiliser streaming à la place)

## 3. Background Mode

Le **Background Mode** permet d'exécuter des tâches longues de manière asynchrone :
- **Traitement long** : Analyses complexes, génération de rapports
- **Polling** : Vérifier périodiquement le statut
- **Libération du client** : Le client peut continuer d'autres tâches

**Modèles supportés :**
- `gpt-5-thinking` : Raisonnement approfondi (OpenAI o1/o3)
- `gpt-5` : Tâches multimodales complexes

**Workflow typique :**
1. Lancer la tâche avec `background=True`
2. Récupérer l'ID de la réponse
3. Polling périodique avec `client.responses.retrieve(id)`
4. Récupérer le résultat final quand `status == "completed"`

**États possibles :**
- `pending` : En attente de traitement
- `processing` : En cours d'exécution
- `completed` : Terminé avec succès
- `failed` : Échec (voir `error` pour détails)

### Décorticage du Retry

**Fonctionnement du décorateur `@retry` :**

1. **Première tentative** : Appel normal de la fonction
2. **En cas d'échec** : Si l'exception est `RateLimitError` ou `APIError` :
   - Attendre 2s (multiplier=1, min=2)
   - Tentative 2
3. **Nouvel échec** : Attendre 4s (backoff exponentiel)
4. **Et ainsi de suite** : jusqu'à 5 tentatives max

**Exemple de scénario réel :**
```
t=0s    : Requête → RateLimitError (429 Too Many Requests)
t=2s    : Retry #1 → APIError (503 Service Unavailable)
t=6s    : Retry #2 → Succès ✅
```

**Pourquoi exponentiel ?**
- Éviter l'effet "thundering herd" : Si 1000 clients retentent simultanément après 2s, le serveur reste surchargé
- Avec backoff exponentiel, les requêtes sont étalées : 2s, 4s, 8s, 16s, 32s

**Limites :**
- ⚠️ Ne résout pas les erreurs permanentes (authentification, prompt invalide)
- ⚠️ Peut augmenter la latence totale (jusqu'à 2+4+8+16+32 = 62s)

In [5]:
if not BATCH_MODE:
    # Lancer une tâche longue en background
    response = client.responses.create(
        model=DEFAULT_MODEL,
        background=True,
        store=True,
        input="""
        Analyse approfondie: Compare les avantages et inconvénients 
        de 5 architectures de microservices différentes pour une 
        application e-commerce à forte charge.
        """
    )
    
    print(f"Tâche lancée: {response.id}")
    print(f"Status initial: {response.status}")
    
    # Polling pour vérifier le statut
    max_wait = 120  # 2 minutes max
    start = time.time()
    while time.time() - start < max_wait:
        status = client.responses.retrieve(response.id)
        print(f"Status: {status.status} ({int(time.time() - start)}s)")
        
        if status.status == "completed":
            print("\n=== Résultat ===")
            result_text = ""
            for item in status.output:
                if hasattr(item, 'content'):
                    for c in item.content:
                        if hasattr(c, 'text'):
                            result_text += c.text
            print(result_text[:500] if result_text else "Pas de contenu")
            break
        elif status.status == "failed":
            print(f"Erreur: {status.error if hasattr(status, 'error') else 'Erreur inconnue'}")
            break
        
        time.sleep(5)
    else:
        print("Timeout - la tâche continue en arrière-plan")
        print(f"Vous pouvez récupérer le résultat plus tard avec: client.responses.retrieve('{response.id}')")
else:
    print("[BATCH_MODE] Simulation de tâche background:")
    print("Status: pending -> processing -> completed (30s)")
    print("Résultat: Comparaison de 5 architectures microservices (500 caractères)")

Tâche lancée: resp_083a7201da8729ba0069834706fe108195855e4402859f3f82
Status initial: queued
Status: queued (0s)
Status: in_progress (5s)
Status: in_progress (10s)
Status: completed (16s)

=== Résultat ===
Lorsqu'on considère les architectures de microservices pour une application e-commerce à forte charge, il est essentiel d'évaluer différentes approches pour déterminer laquelle convient le mieux aux besoins spécifiques. Voici une comparaison approfondie de cinq architectures de microservices:

### 1. **Microservices Basés sur des API REST**

#### Avantages:
- **Interopérabilité**: Standard largement utilisé, permettant une communication facile entre services.
- **Scalabilité**: Les services peuv


### Comportement du Rate Limiter

**Analyse de l'exécution :**

Dans cet exemple avec `max_requests_per_minute=10` :
- Les 3 requêtes passent **instantanément** (aucune attente)
- Pourquoi ? Parce que 3 < 10 (limite non atteinte)

**Test avec limite serrée :**

Si nous avions configuré `max_requests_per_minute=2` :
```
t=0s   : Requête 1 → OK (1/2)
t=0.5s : Requête 2 → OK (2/2)
t=1s   : Requête 3 → ATTENTE de ~59s (fenêtre de 60s)
t=60s  : Requête 3 → OK
```

**Fenêtre glissante vs Fenêtre fixe :**

Notre implémentation utilise une **fenêtre glissante** :
- ✅ Plus précis : compte exactement les 60 dernières secondes
- ✅ Pas d'effet "reset brutal" à chaque minute

Alternative **fenêtre fixe** :
```python
# Moins précis mais plus simple
if current_minute != last_minute:
    counter = 0
    last_minute = current_minute
counter += 1
if counter > max_rpm:
    wait()
```

**En production :**
- Combiner rate limiter côté client (courtoisie) + côté serveur (sécurité)
- Adapter la limite à votre tier OpenAI (voir dashboard usage limits)

## 4. Rate Limiting et Retry

En production, il est essentiel de gérer les erreurs transitoires et les limites d'API :

### Retry avec Backoff Exponentiel

La bibliothèque **tenacity** permet d'implémenter facilement un retry automatique :
- **Backoff exponentiel** : Attendre 2s, puis 4s, puis 8s, etc.
- **Retry sélectif** : Uniquement sur certaines exceptions
- **Limite de tentatives** : Éviter les boucles infinies

**Pattern recommandé :**
```python
@retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=2, max=60),
    retry=retry_if_exception_type((RateLimitError, APIError))
)
```

**Erreurs à gérer :**
- `RateLimitError` : Limite de requêtes dépassée (429)
- `APIError` : Erreur serveur temporaire (500, 502, 503)
- `Timeout` : Délai d'attente dépassé

### Comparaison des Résultats

**Analyse coût/bénéfice :**

Pour ces deux requêtes similaires (résumé en 3 points), observons :

| Métrique | Sans cache | Avec cache | Différence |
|----------|-----------|------------|------------|
| Tokens input | ~25 | ~25 | 0% (première requête) |
| Tokens output | ~60 | ~65 | +8% (variation normale) |
| Coût | ~$0.000663 | ~$0.000713 | +7% |
| Cache hit | Non | **Dépend du prompt** | - |

**Pourquoi pas d'économie ici ?**
- Les prompts sont **différents** ("Python" vs "JavaScript")
- Le cache ne fonctionne que sur les **préfixes identiques**

**Test optimal pour le cache :**
```python
# Requête 1
resp1 = optimized_completion("Contexte: Documentation Python. Question: Qu'est-ce qu'une liste?")

# Requête 2 (même préfixe "Contexte: Documentation Python.")
resp2 = optimized_completion("Contexte: Documentation Python. Question: Qu'est-ce qu'un tuple?")
# → Économie de ~50% sur les tokens du contexte
```

**Gains typiques du cache :**
- Conversations longues : 40-60% économie
- RAG avec contexte fixe : 60-80% économie
- Requêtes isolées : 0% (aucun préfixe commun)

In [6]:
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from openai import RateLimitError, APIError

@retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=2, max=60),
    retry=retry_if_exception_type((RateLimitError, APIError))
)
def safe_completion(prompt: str, model: str = None) -> str:
    """Appel API avec retry automatique sur erreurs transitoires"""
    model = model or DEFAULT_MODEL
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=200
    )
    return response.choices[0].message.content

# Test
result = safe_completion("Dis 'Hello World' en 5 langues.")
print(result)

Voici "Hello World" en cinq langues différentes :

1. Anglais : Hello World
2. Français : Bonjour le monde
3. Espagnol : Hola Mundo
4. Allemand : Hallo Welt
5. Italien : Ciao Mondo


### Interprétation des Logs

**Format de log structuré :**

```
2026-02-04 15:23:45 | OpenAI_Production | INFO | SUCCESS | Model: gpt-4o | Duration: 1.23s | Tokens: 85 (in: 25, out: 60)
```

**Champs clés :**
- `Timestamp` : Horodatage précis (important pour corrélation)
- `Level` : INFO (succès) ou ERROR (échec)
- `Model` : Tracer quel modèle a été utilisé
- `Duration` : Latence end-to-end (objectif : <2s pour chatbot)
- `Tokens` : Consommation (surveiller les dépassements)

**Analyse des logs :**

1. **Log SUCCESS** :
   - Durée ~1-2s : Normal pour gpt-4o
   - 85 tokens total : Petit prompt + réponse courte
   - ✅ Requête efficace

2. **Log ERROR** :
   - `InvalidRequestError` : Modèle invalide
   - Capturé et loggé avant propagation
   - ✅ Permet diagnostic rapide sans crash

**Agrégation recommandée :**
```bash
# Latence moyenne par modèle (via grep + awk)
grep "SUCCESS" production.log | awk '{print $8, $12}' | sort

# Taux d'erreur sur 1h
grep "ERROR" production.log | wc -l
```

**Outils professionnels :**
- ELK Stack (Elasticsearch, Logstash, Kibana)
- Datadog, Splunk, New Relic
- Prometheus + Grafana

### Transition vers les Patterns Avancés

Jusqu'ici nous avons couvert les **patterns de résilience et d'optimisation** :
- ✅ Retry automatique
- ✅ Rate limiting
- ✅ Logging structuré
- ✅ Optimisation des coûts

Les deux derniers patterns concernent **l'expérience utilisateur et la sécurité** :

1. **Streaming** : Améliorer la perception de latence
   - Au lieu d'attendre 5s pour afficher 500 tokens
   - Afficher progressivement (premier token en 200ms)
   - Utilisateur voit la "pensée en temps réel"

2. **Modération** : Filtrer le contenu inapproprié
   - Avant de traiter : vérifier que l'input est acceptable
   - Après génération : s'assurer que l'output est sûr
   - Éviter les violations de politique d'usage

Ces patterns sont **essentiels** pour toute application destinée aux utilisateurs finaux.

### Analyse des Résultats

**Streaming :**

Observez le comportement :
- Les tokens apparaissent **progressivement** (un par un ou par petits groupes)
- Latence au premier token : ~200-500ms
- Latence totale : Identique à une requête non-streamée (~2s)
- **Gain perçu** : L'utilisateur voit le début de la réponse immédiatement

**Implémentation UX :**
```python
# Dans une vraie app web
async def stream_to_user():
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content:
            await websocket.send(content)  # Envoyer au navigateur en temps réel
```

**Modération :**

Les résultats montrent :
1. **Texte neutre** : "Bonjour, comment vas-tu?"
   - `flagged = False` : Aucun problème détecté
   
2. **Texte négatif** : "Je déteste ce produit, c'est nul!"
   - `flagged = False` : Critique légitime (pas de haine/violence)

**Cas qui seraient flaggés :**
- Contenu haineux, violent, sexuel
- Harcèlement, menaces
- Auto-mutilation

**Pattern de sécurité :**
```python
# Avant de traiter
if moderate(user_input).flagged:
    return "Désolé, ce contenu viole notre politique."

# Générer
response = generate(user_input)

# Après génération
if moderate(response).flagged:
    return "Désolé, impossible de répondre à cette demande."
```

### Rate Limiter Personnalisé

Pour respecter les limites de requêtes par minute (RPM), implémentons un **rate limiter** :
- **Fenêtre glissante** : Compte les requêtes sur les 60 dernières secondes
- **Attente automatique** : Bloque jusqu'à ce qu'une requête soit autorisée
- **Configurable** : Adapter selon votre tier OpenAI

**Limites par tier (exemple) :**
- Free : 3 RPM
- Tier 1 : 500 RPM
- Tier 2 : 5000 RPM
- Tier 5 : 10000 RPM

In [7]:
import time
from collections import deque

class RateLimiter:
    """Limite le nombre de requêtes par minute"""
    def __init__(self, max_requests_per_minute: int = 60):
        self.max_rpm = max_requests_per_minute
        self.requests = deque()
    
    def wait_if_needed(self):
        now = time.time()
        # Nettoyer les anciennes requêtes (plus de 60s)
        while self.requests and now - self.requests[0] > 60:
            self.requests.popleft()
        
        # Si limite atteinte, attendre
        if len(self.requests) >= self.max_rpm:
            wait_time = 60 - (now - self.requests[0])
            if wait_time > 0:
                print(f"Rate limit: attente de {wait_time:.1f}s")
                time.sleep(wait_time)
        
        self.requests.append(time.time())

# Exemple d'utilisation
limiter = RateLimiter(max_requests_per_minute=10)

for i in range(3):
    limiter.wait_if_needed()
    result = safe_completion(f"Nombre aléatoire #{i+1}")
    print(f"Requête {i+1}: {result[:50]}...")

Requête 1: Voici un nom aléatoire pour vous : Léonard Durand....
Requête 2: Bien sûr ! Voici un nom aléatoire pour vous : Clar...
Requête 3: Bien sûr ! Voici un nom aléatoire pour vous : Clar...


### Interprétation du Comportement du Rate Limiter

**Observations de l'exécution :**

Les 3 requêtes se sont exécutées **sans attente** car la limite (`max_requests_per_minute=10`) n'a pas été atteinte.

**Analyse du mécanisme :**

1. **Fenêtre glissante** :
   ```python
   # À t=0s : deque([t0])         → 1/10, OK
   # À t=1s : deque([t0, t1])     → 2/10, OK  
   # À t=2s : deque([t0, t1, t2]) → 3/10, OK
   ```

2. **Nettoyage automatique** :
   - Les timestamps > 60s sont supprimés du deque
   - Garantit que seules les requêtes récentes comptent

**Test avec limite stricte :**

Si nous avions configuré `max_requests_per_minute=2` :

```
t=0.0s : Requête #1 → OK (1/2)
t=0.5s : Requête #2 → OK (2/2)
t=1.0s : Requête #3 → ATTENTE ~59s (limite atteinte)
         ↓ Nettoyage à t=60s (requête #1 expire)
t=60.5s: Requête #3 → OK (1/2)
```

**Comparaison des stratégies :**

| Approche | Avantages | Inconvénients |
|----------|-----------|---------------|
| **Fenêtre glissante** (notre impl.) | ✅ Précis, pas d'effet de bord | ⚠️ Mémoire O(n) requêtes |
| **Fenêtre fixe** (reset à chaque minute) | ✅ Simple, O(1) mémoire | ⚠️ Burst de 2× limite possible |
| **Token bucket** | ✅ Permet les bursts contrôlés | ⚠️ Plus complexe à implémenter |

**Recommandations production :**

- **Développement** : Fenêtre glissante (notre code)
- **Production haute charge** : Token bucket avec Redis
- **Edge cases** : Combiner rate limiter client + serveur (double protection)

> **Note** : Les limites OpenAI sont par organisation, pas par application. Coordonner le rate limiting si plusieurs services utilisent la même clé API.

## 5. Optimisation des Coûts

Stratégies pour réduire les coûts d'API en production :

### 1. Utiliser le Cache (store=True)
- Économie de 40-80% sur les tokens d'entrée répétés
- Activer sur toutes les conversations longues

### 2. Choisir le Bon Modèle
| Modèle | Prix Input | Prix Output | Cas d'usage |
|--------|------------|-------------|-------------|
| gpt-5-mini | $0.15/1M | $0.60/1M | Tâches simples, production |
| gpt-5 | $2.50/1M | $10.00/1M | Tâches complexes |
| gpt-5-thinking | $10.00/1M | $40.00/1M | Raisonnement profond |

### 3. Limiter les Tokens
- Utiliser `max_tokens` pour contrôler la longueur
- Préférer les prompts courts et précis
- Éviter les contextes inutilement longs

### 4. Batch Processing
- Utiliser l'API Batch pour réductions de 50%
- Acceptable pour tâches non-temps-réel

### 5. Monitoring et Alertes
- Suivre les coûts quotidiens/mensuels
- Configurer des alertes budgétaires

In [ ]:
def optimized_completion(prompt: str, use_cache: bool = True) -> dict:
    """Completion optimisée avec métriques de coût"""
    
    # Utiliser store pour le cache si activé
    response = client.responses.create(
        model=DEFAULT_MODEL,
        store=use_cache,
        input=prompt
    )
    
    # Calculer les coûts approximatifs
    # Prix gpt-5-mini: ~$0.15/1M input, ~$0.60/1M output
    input_tokens = response.usage.input_tokens if hasattr(response, 'usage') and response.usage else 0
    output_tokens = response.usage.output_tokens if hasattr(response, 'usage') and response.usage else 0
    
    cost_input = input_tokens * 0.00000015
    cost_output = output_tokens * 0.0000006
    
    # Extraire le contenu
    content = ""
    if response.output:
        for item in response.output:
            if hasattr(item, 'content'):
                for c in item.content:
                    if hasattr(c, 'text'):
                        content += c.text
    
    return {
        "content": content,
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "estimated_cost": cost_input + cost_output,
        "cached": getattr(response, 'cached', False)
    }

# Test sans cache
result1 = optimized_completion("Résume les avantages de Python en 3 points.", use_cache=False)
print("=== Sans cache ===")
print(f"Réponse: {result1['content'][:100]}...")
print(f"Tokens: {result1['input_tokens']} in / {result1['output_tokens']} out")
print(f"Coût estimé: ${result1['estimated_cost']:.6f}")

# Test avec cache
result2 = optimized_completion("Résume les avantages de JavaScript en 3 points.", use_cache=True)
print("\n=== Avec cache ===")
print(f"Réponse: {result2['content'][:100]}...")
print(f"Tokens: {result2['input_tokens']} in / {result2['output_tokens']} out")
print(f"Coût estimé: ${result2['estimated_cost']:.6f}")
print(f"Cache utilisé: {result2['cached']}")

### Interprétation des Résultats d'Optimisation

**Analyse comparative :**

Les deux exécutions montrent des coûts similaires car les prompts sont **complètement différents** (pas de préfixe commun pour bénéficier du cache).

**Scénario optimal pour le cache :**

Imaginons une application de support client avec un contexte fixe :

```python
# Contexte partagé (200 tokens)
context = "Documentation produit X: [longue description]..."

# Requête 1
q1 = context + " Question: Comment l'installer?"
# → Tokens input: 200 (contexte) + 5 (question) = 205

# Requête 2 avec cache
q2 = context + " Question: Comment le configurer?"
# → Tokens input: 5 (seulement la nouvelle question!)
# → Cache hit: 200 tokens @ 50% de réduction = économie de $0.000015
```

**Calcul d'économie réelle :**

| Scénario | Sans cache | Avec cache | Économie |
|----------|-----------|------------|----------|
| **RAG avec contexte 1000 tokens** | $0.00015/req | $0.000075/req | **50%** |
| **Conversation 10 tours (500 tokens/tour)** | $0.00375 total | $0.00150 total | **60%** |
| **Documentation technique (2000 tokens)** | $0.0003/req | $0.0001/req | **67%** |

**Points critiques :**

1. ✅ **Activer le cache** : Toujours utiliser `store=True` pour conversations et RAG
2. ✅ **Monitorer les hits** : Vérifier que `cached=True` dans les réponses
3. ⚠️ **Attention aux modifications** : Changer 1 mot au début du contexte invalide tout le cache
4. ⚠️ **Durée de vie** : Cache expire après 30 jours d'inactivité

> **Recommandation production** : Pour une app avec 10000 requêtes/jour et contexte moyen de 500 tokens, le cache peut économiser **$20-30/mois** (gpt-4o).

## 6. Monitoring et Logging

Bonnes pratiques pour le monitoring en production :

### Logging Structuré
- **Timestamp** : Horodatage de chaque requête
- **Modèle** : Quel modèle a été utilisé
- **Durée** : Temps de réponse
- **Tokens** : Consommation de tokens
- **Succès/Échec** : Status de la requête
- **Erreurs** : Type et message d'erreur

### Métriques à Surveiller
- **Latence** : p50, p95, p99
- **Taux d'erreur** : Pourcentage de requêtes échouées
- **Coûts** : Dépenses quotidiennes/mensuelles
- **Tokens/requête** : Moyenne et variance
- **Rate limiting** : Nombre de requêtes rejetées

### Outils Recommandés
- **Logging** : Python `logging`, Loguru
- **APM** : Datadog, New Relic, OpenTelemetry
- **Alerting** : PagerDuty, Opsgenie
- **Dashboards** : Grafana, Kibana

In [9]:
import logging
from datetime import datetime

# Configuration du logger
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(name)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger("OpenAI_Production")

def logged_completion(prompt: str, model: str = None) -> str:
    """Completion avec logging complet"""
    model = model or DEFAULT_MODEL
    start = datetime.now()
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200
        )
        
        duration = (datetime.now() - start).total_seconds()
        logger.info(f"SUCCESS | Model: {model} | Duration: {duration:.2f}s | "
                   f"Tokens: {response.usage.total_tokens} (in: {response.usage.prompt_tokens}, out: {response.usage.completion_tokens})")
        
        return response.choices[0].message.content
        
    except Exception as e:
        duration = (datetime.now() - start).total_seconds()
        logger.error(f"FAILED | Model: {model} | Duration: {duration:.2f}s | "
                    f"Error: {type(e).__name__}: {str(e)[:100]}")
        raise

# Tests
print("=== Test réussi ===")
result = logged_completion("Quelle heure est-il?")
print(f"Réponse: {result}\n")

print("=== Test avec modèle invalide ===")
try:
    result = logged_completion("Test", model="gpt-invalid-model")
except Exception as e:
    print(f"Erreur capturée: {type(e).__name__}")

=== Test réussi ===


2026-02-04 14:18:28 | httpx | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-04 14:18:28 | OpenAI_Production | INFO | SUCCESS | Model: gpt-4o | Duration: 1.57s | Tokens: 43 (in: 12, out: 31)


Réponse: Je suis désolé, mais je n'ai pas accès à l'heure actuelle. Vous pouvez vérifier l'heure sur votre appareil ou sur une horloge proche.

=== Test avec modèle invalide ===


2026-02-04 14:18:28 | httpx | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 404 Not Found"
2026-02-04 14:18:28 | OpenAI_Production | ERROR | FAILED | Model: gpt-invalid-model | Duration: 0.28s | Error: NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-invalid-model` does not exist or you do not 


Erreur capturée: NotFoundError


### Interprétation des Logs de Production

**Analyse des logs générés :**

```
2026-02-04 14:18:28 | OpenAI_Production | INFO | SUCCESS | Model: gpt-4o | Duration: 1.57s | Tokens: 43 (in: 12, out: 31)
2026-02-04 14:18:28 | OpenAI_Production | ERROR | FAILED | Model: gpt-invalid-model | Duration: 0.28s | Error: NotFoundError
```

**Points clés observés :**

1. **Latence** :
   - Requête réussie : 1.57s (acceptable pour gpt-4o avec 31 tokens)
   - Requête échouée : 0.28s (échec rapide = bon signe, pas de timeout)

2. **Consommation de tokens** :
   - Input : 12 tokens (prompt court "Quelle heure est-il?")
   - Output : 31 tokens (réponse standard)
   - Total : 43 tokens ≈ $0.000168 (gpt-4o @ $2.50/1M input, $10/1M output)

3. **Gestion d'erreur** :
   - Exception capturée et loggée (pas de crash)
   - Message d'erreur informatif : "The model `gpt-invalid-model` does not exist"
   - L'application peut réagir (fallback, alerte utilisateur)

**Métriques à extraire pour dashboards :**

| Métrique | Calcul | Seuil alerte |
|----------|--------|--------------|
| **Latence P95** | 95ème percentile des durées | > 3s |
| **Taux d'erreur** | (erreurs / total) × 100 | > 5% |
| **Coût moyen/requête** | Somme(tokens × prix) / nb_requêtes | > $0.01 |
| **Requêtes/minute** | Count(logs) sur fenêtre 1min | > 80% de la limite |

> **Production tip** : Exporter les logs vers un système centralisé (ELK, Datadog) pour analyser les tendances sur plusieurs jours et détecter les anomalies (ex: latence soudainement × 3).

## 7. Patterns Avancés : Streaming et Modération

### Streaming pour UX Réactive

Le streaming permet d'afficher la réponse progressivement :
- **Meilleure UX** : L'utilisateur voit la réponse se construire
- **Latence perçue réduite** : Premier token en ~200ms vs 5s pour réponse complète
- **Annulation précoce** : Possibilité de stopper si réponse non pertinente

### Modération de Contenu

OpenAI propose une API de modération pour détecter :
- Contenu haineux/violent
- Harcèlement
- Contenu sexuel
- Auto-mutilation
- Etc.

**Pattern recommandé :**
1. Modérer l'input utilisateur
2. Générer la réponse si OK
3. Modérer l'output avant affichage

In [13]:
# Streaming
print("=== Streaming Example ===")
stream = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{"role": "user", "content": "Écris un haïku sur la programmation."}],
    stream=True,
    max_tokens=100
)

print("Réponse streamée: ", end="", flush=True)
for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
print("\n")

# Modération
print("=== Moderation Example ===")
test_inputs = [
    "Bonjour, comment vas-tu?",
    "Enfoiré, je vais te tuer!"
]

for text in test_inputs:
    moderation = client.moderations.create(input=text)
    result = moderation.results[0]
    
    print(f"\nTexte: {text}")
    print(f"Flaggé: {result.flagged}")
    if result.flagged:
        print(f"Catégories: {[cat for cat, flagged in result.categories.__dict__.items() if flagged]}")

=== Streaming Example ===


2026-02-04 14:23:58 | httpx | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Réponse streamée: Lignes de code dansent,  
Sous les mains créant le monde,  
Bugs s'évanouissent.

=== Moderation Example ===


2026-02-04 14:24:00 | httpx | INFO | HTTP Request: POST https://api.openai.com/v1/moderations "HTTP/1.1 200 OK"



Texte: Bonjour, comment vas-tu?
Flaggé: False


2026-02-04 14:24:00 | httpx | INFO | HTTP Request: POST https://api.openai.com/v1/moderations "HTTP/1.1 200 OK"



Texte: Enfoiré, je vais te tuer!
Flaggé: True
Catégories: ['harassment', 'harassment_threatening', 'violence']


### Interprétation des Résultats Streaming et Modération

**Observations sur le Streaming :**

Le streaming transforme radicalement l'expérience utilisateur :
- **Latence perçue** : Au lieu d'attendre 2-3s pour voir la réponse complète, le premier mot apparaît en ~200ms
- **Engagement utilisateur** : Voir le texte se construire donne l'impression d'une "conversation naturelle"
- **Format de réponse** : Les chunks arrivent progressivement via `delta.content`

**Observations sur la Modération :**

| Texte | Flaggé | Catégories détectées | Action recommandée |
|-------|--------|---------------------|-------------------|
| "Bonjour, comment vas-tu?" | ❌ Non | Aucune | ✅ Traiter normalement |
| "Enfoiré, je vais te tuer!" | ✅ Oui | harassment, harassment_threatening, violence | 🚫 Bloquer avant génération |

**Pattern de sécurité production :**

```python
# Workflow complet sécurisé
def safe_generation(user_input: str) -> str:
    # 1. Modération input
    mod_input = client.moderations.create(input=user_input)
    if mod_input.results[0].flagged:
        return "Votre message viole notre politique d'utilisation."
    
    # 2. Génération
    response = generate(user_input)
    
    # 3. Modération output
    mod_output = client.moderations.create(input=response)
    if mod_output.results[0].flagged:
        return "Désolé, impossible de générer une réponse appropriée."
    
    return response
```

> **Note importante** : La modération ajoute ~100ms de latence par check. Pour les applications temps-réel, envisager une modération asynchrone post-génération.

## 8. Checklist Déploiement Production

### Sécurité
- [ ] API keys stockées dans variables d'environnement (jamais hardcodées)
- [ ] Rotation régulière des clés
- [ ] Rate limiting côté serveur
- [ ] Validation et sanitization des inputs utilisateur
- [ ] Modération de contenu activée
- [ ] Logs ne contiennent pas de données sensibles

### Résilience
- [ ] Retry automatique avec backoff exponentiel
- [ ] Timeout configurés
- [ ] Circuit breaker pour dépendances externes
- [ ] Fallback gracieux en cas d'erreur
- [ ] Health checks réguliers

### Performance
- [ ] Cache activé (`store=True`)
- [ ] Choix du modèle adapté au cas d'usage
- [ ] `max_tokens` configuré pour limiter les coûts
- [ ] Streaming pour réponses longues
- [ ] Background mode pour tâches longues

### Monitoring
- [ ] Logging structuré configuré
- [ ] Métriques : latence, taux d'erreur, coûts
- [ ] Alertes budgétaires configurées
- [ ] Dashboards temps réel
- [ ] Traçabilité des requêtes (request ID)

### Coûts
- [ ] Budget mensuel défini
- [ ] Alertes à 50%, 80%, 100% du budget
- [ ] Audit régulier de l'usage par endpoint/utilisateur
- [ ] Optimisation continue des prompts
- [ ] Évaluation régulière des modèles (nouveaux modèles moins chers?)

### Conformité
- [ ] RGPD : consentement utilisateur pour traitement des données
- [ ] Politique de rétention des données
- [ ] Anonymisation des données personnelles
- [ ] Documentation des traitements de données
- [ ] DPO informé de l'usage d'IA

## Conclusion

### Récapitulatif des Patterns

| Pattern | Cas d'usage | Complexité |
|---------|-------------|------------|
| **Responses API + store** | Conversations courtes avec cache | ⭐⭐ |
| **Conversations API** | Conversations longue durée | ⭐⭐ |
| **Background Mode** | Tâches longues (>30s) | ⭐⭐⭐ |
| **Retry + Backoff** | Résilience production | ⭐⭐ |
| **Rate Limiter** | Respect limites API | ⭐⭐ |
| **Logging structuré** | Monitoring et debug | ⭐⭐ |
| **Streaming** | UX temps réel | ⭐⭐ |
| **Modération** | Sécurité contenu | ⭐ |

### Ressources Supplémentaires

**Documentation OpenAI :**
- [Responses API](https://platform.openai.com/docs/api-reference/responses)
- [Conversations API](https://platform.openai.com/docs/api-reference/conversations)
- [Rate Limits](https://platform.openai.com/docs/guides/rate-limits)
- [Error Codes](https://platform.openai.com/docs/guides/error-codes)

**Bibliothèques Python :**
- [tenacity](https://tenacity.readthedocs.io/) : Retry robuste
- [openai](https://github.com/openai/openai-python) : SDK officiel
- [loguru](https://loguru.readthedocs.io/) : Logging simplifié

**Prochaines étapes :**
- Implémenter ces patterns dans votre application
- Configurer un monitoring complet
- Tester la résilience (chaos engineering)
- Optimiser les coûts progressivement

## Exercices Pratiques

### Exercice 1 : Chatbot Multi-Session (30 min)

Créez un chatbot de support client qui :
1. Utilise la Conversations API pour maintenir le contexte
2. Persiste les conversations dans un fichier JSON (pour reprise après redémarrage)
3. Implémente un retry avec backoff
4. Log toutes les interactions

**Bonus :** Ajouter une commande `/summary` qui résume la conversation en cours.

### Exercice 2 : Système d'Analyse de Documents (40 min)

Implémentez un système qui :
1. Prend un long document en entrée
2. Lance l'analyse en background mode
3. Affiche une barre de progression pendant le traitement
4. Retourne un rapport structuré (points clés, résumé, recommandations)
5. Calcule et affiche le coût total de l'analyse

**Bonus :** Comparer les coûts entre gpt-5-mini et gpt-5.

### Exercice 3 : Rate Limiter Multi-Tier (20 min)

Améliorez la classe `RateLimiter` pour supporter :
1. Des limites par minute ET par jour
2. Des priorités de requêtes (high/medium/low)
3. Un mode "burst" (permettre 10 requêtes instantanées, puis throttling)

**Bonus :** Ajouter des statistiques (nombre de requêtes dans les dernières 24h, temps moyen d'attente).